<a href="https://colab.research.google.com/github/Mohneesh09/AMEX-Challenge/blob/main/Trend_Channel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will contain all research and experiments with trend and channel prediction algos

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
API_KEY = "717311313634cf72e5c3b021728afe4e"

In [6]:
from urllib.request import urlopen 
import json
ticker='SPY'
url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey={}".format(ticker, API_KEY)
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
data_json = json.loads(response.read())

print(data_json)

{'symbol': 'SPY', 'historical': [{'date': '2023-02-13', 'open': 408.72, 'high': 412.76, 'low': 408.24, 'close': 412.575, 'adjClose': 412.575, 'volume': 33982514, 'unadjustedVolume': 33982514, 'change': 3.85, 'changePercent': 0.94318849, 'vwap': 411.19, 'label': 'February 13, 23', 'changeOverTime': 0.0094318849}, {'date': '2023-02-10', 'open': 405.86, 'high': 408.44, 'low': 405.02, 'close': 408.04, 'adjClose': 408.04, 'volume': 70776210, 'unadjustedVolume': 70769715, 'change': 2.18, 'changePercent': 0.537, 'vwap': 407.16667, 'label': 'February 10, 23', 'changeOverTime': 0.00537}, {'date': '2023-02-09', 'open': 414.41, 'high': 414.57, 'low': 405.81, 'close': 407.09, 'adjClose': 407.09, 'volume': 78443501, 'unadjustedVolume': 78438000, 'change': -7.32, 'changePercent': -1.766, 'vwap': 409.15667, 'label': 'February 09, 23', 'changeOverTime': -0.01766}, {'date': '2023-02-08', 'open': 413.13, 'high': 414.53, 'low': 409.93, 'close': 410.65, 'adjClose': 410.65, 'volume': 76234805, 'unadjustedV

In [8]:
df=pd.DataFrame(data_json['historical'])
df=df[['date','close','open','high','low']]

In [26]:
df1=df.set_index('date')

In [ ]:
len(df)
#df.head()

1258

In [10]:
df.index

RangeIndex(start=0, stop=1258, step=1)

In [19]:
#https://medium.com/@chris_42047/3-ways-to-smooth-the-price-waves-with-python-9a4e170aaf15
#Will use Nadaraya–Watson kernel regression for smoothing prices.
from statsmodels.nonparametric.kernel_regression import KernelReg
def smoothed_curve(df_close,bandwidth):
  kernel_reg= KernelReg([df_close.values],[df_close.index],var_type='c',bw=bandwidth)
  reg_result = kernel_reg.fit([df_close.index])
  smoothed_df_close = pd.Series(data=reg_result[0], index=df_close.index)

  return smoothed_df_close


In [17]:
from scipy.signal import argrelextrema
def find_extrema(df_close):
    #Calculating local min and max
    smoothed_local_max = argrelextrema(df_close.values, np.greater, order=12)[0]
    smoothed_local_min = argrelextrema(df_close.values, np.less, order=12)[0]

    smoothed_max_list = []
    smoothed_min_list = []
    max_id = 0
    min_id = 0
    for i in range(0, len(df_close), 1):
        if max_id < len(smoothed_local_max):
            if i == smoothed_local_max[max_id]:
                smoothed_max_list.append(df_close[i])
                max_id += 1
            else:
                smoothed_max_list.append(np.nan)
        if min_id < len(smoothed_local_min):
            if i == smoothed_local_min[min_id]:
                smoothed_min_list.append(df_close[i])
                min_id += 1
            else:
                smoothed_min_list.append(np.nan)

    return smoothed_max_list, smoothed_min_list

In [39]:
def cal_trendlines(values):
    if len(values) == 0:
        return 0, 0
    x = np.arange(1, len(values) + 1, 1)
    y = np.array(values)

    x_new = x[~np.isnan(y)]
    y_new = y[~np.isnan(y)]
    m, c = np.polyfit(x_new, y_new, 1)
    return m, c     #Slope and Intercept